ANTsPy and ANTsR inherit the ability to do multi-metric registration.

Such registrations assume that the feature images are in the same physical space and at least roughly aligned.

One may then use registration to optimize a multiple similarity metric objective function as in:

https://doi.org/10.3389/fninf.2014.00044

https://www.ncbi.nlm.nih.gov/pubmed/18995188

First, import ants, read some images and create some features.

In [1]:
import ants
image = ants.image_read(ants.get_ants_data('r16'))
image2 = ants.image_read(ants.get_ants_data('r27'))
g1 = ants.iMath_grad( image )
g2 = ants.iMath_grad( image2 )

Perform a baseline registration with a single feature and create a couple of new metrics.

In [2]:
reg1 = ants.registration( image, image2, 'SyNOnly' )
demonsMetric = ['demons', g1, g2, 1, 1]
ccMetric = ['CC', image, image2, 2, 1 ]